In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# Các hàm xử lý phụ 

# Hàm xử lý ngày tháng năm
def parse_date(date_text):
    try:
        date_obj = datetime.strptime(date_text, '%d/%m/%Y')
        return date_obj.day, date_obj.month, date_obj.year
    except ValueError:
        print(f"Unable to parse date: {date_text}")
        return None, None, None
    
# Hàm để xử lý giá trị rỗng
def process_value(value, is_numeric=False):
    if value in [None, 'nan', 'NaN', 'N/A', '']:
        return 0 if is_numeric else "NOT FOUND"
    return value    

# Hàm chuyển đổi giá trị thành số
def convert_to_number(value):
    if isinstance(value, (int, float)):
        return value
    if isinstance(value, str):
        value = ''.join(filter(str.isdigit, value))
        return int(value) if value else 0
    return 0

In [3]:
def get_novel_html(novel_id):
    url = f"https://ln.hako.vn/truyen/{novel_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise exception for bad status codes
        return response.text
    except requests.RequestException as e:
        print(f"Error fetching novel {novel_id}: {str(e)}")
        return None

In [4]:
# Các hàm trích xuất thông tin

# Trích xuất ID
def extract_id_from_url(url):
    patterns = [
        r'/truyen/(\d+)',
        r'/sang-tac/(\d+)-',
        r'/convert/(\d+)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

# Trích xuất thông tin 
def extract_info_from_html(novel_id, html_content):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        title = soup.title.string.strip() if soup.title else "NOT FOUND"
        title = re.sub(r' - Cổng Light Novel - Đọc Light Novel$', '', title) 
            
        # Trích xuất link
        canonical_link = soup.find('link', rel='canonical')
        canonical_url = canonical_link['href'] if canonical_link else "NOT FOUND"
        canonical_url = canonical_url.replace('docln.net', 'ln.hako.vn')
        url_id = extract_id_from_url(canonical_url)  # Trích xuất ID từ URL

        # Trích xuất phương thức sáng tác
        method = "NOT FOUND"
        method_span = soup.find('div', class_='series-type')
        if method_span:
            method_link = method_span.find('span')
        if method_link and method_link.string:
            method = method_link.string.strip()

        # Trích xuất thông tin về thể loại
        genres = []
        manga = "Not sure"
        anime = "Not sure"
        cd = "Not sure"
        origin = "vietnamese" if method == "Truyện sáng tác" else "japanese"
        genre_items = soup.find_all(class_='series-gerne-item')
        for item in genre_items:
            genre_text = item.get_text(strip=True)
            if "Manga" in genre_text: manga = "Yes"
            if "Anime" in genre_text: anime = "Yes"
            if "CD" in genre_text: cd = "Yes"
            if "Chinese" in genre_text: origin = "chinese"
            if "English" in genre_text: origin = "english"
            if "Korean" in genre_text: origin = "korean"
            elif not any(word in genre_text for word in ["Manga", "Anime", "CD", "Chinese", "English", "Korean"]):
                genres.append(genre_text)

        # Trích xuất link ảnh
        image_link = "NOT FOUND"
        content_div = soup.find('div', class_='content img-in-ratio')
        if content_div and 'style' in content_div.attrs:
            style = content_div['style']
            match = re.search(r"url\('([^']+)'\)", style)
            if match:
                image_link = match.group(1)
        
        # Trích xuất tác giả
        author = "NOT FOUND"
        author_span = soup.find('span', class_='info-name', string='Tác giả:')
        if author_span:
            info_value_span = author_span.find_next_sibling('span', class_='info-value')
            if info_value_span:
                author_link = info_value_span.find('a')
            if author_link:
                author = author_link.string.strip()
    
        # Trích xuất họa sĩ
        artist = "NOT FOUND"
        artist_span = soup.find('span', class_='info-name', string='Họa sĩ:')
        if artist_span:
            info_value_span = artist_span.find_next_sibling('span', class_='info-value')
            if info_value_span and info_value_span.string:
                artist = info_value_span.string.strip()

        # Trích xuất kiểu trình bày
        showtype = 'web novel' if method == "Truyện sáng tác" or artist.lower() in ['NOT FOUND', 'N/A'] else 'light novel'

        # Trích xuất tình trạng
        state = "NOT FOUND"
        state_span = soup.find('span', class_='info-name', string='Tình trạng:')
        if state_span:
            info_value_span = state_span.find_next_sibling('span', class_='info-value')
            if info_value_span:
                state_link = info_value_span.find('a')
                if state_link:
                    state = state_link.string.strip() 

        # Trích xuất số like
        like = 0
        like_span = soup.find('span', class_='block feature-value')
        if like_span:
            like_link = like_span.find_next_sibling('span', class_='block feature-name')
            if like_link and like_link.string:
                like_text = like_link.string.strip()
                try:
                    like = float(like_text)
                except ValueError:
                    pass

        # Trích xuất số từ
        nword = 0
        nword_span = soup.find('div', class_='statistic-name', string='Số từ')
        if nword_span:
            nword_link = nword_span.find_next_sibling('div', class_='statistic-value')
            if nword_link:
                nword = nword_link.string.strip()

        # Trích xuất số lượt đánh giá
        rate = 0  
        rate_span = soup.find('div', class_='statistic-name', string='Đánh giá')
        if rate_span:
            rate_link = rate_span.find_next_sibling('div', class_='statistic-value')
            if rate_link and rate_link.string:
                rate_text = rate_link.string.strip()
                try:
                    rate = float(rate_text)
                except ValueError:
                    pass

        # Trích xuất số lượt xem
        view = 0
        view_span = soup.find('div', class_='statistic-name', string='Lượt xem')
        if view_span:
            view_link = view_span.find_next_sibling('div', class_='statistic-value')
            if view_link:
                view = view_link.string.strip()

        # Trích xuất số lượt bình luận
        ncom = 0
        ncom_span = soup.find('span', class_='comments-count')
        if ncom_span:
            ncom = ncom_span.string.strip()
            ncom = re.sub(r'[()]', '', ncom)

        # Trích xuất tên gọi khác
        fname = None
        fname_span = soup.find('div', class_='fact-value')
        if fname_span:
            fname_links = fname_span.find_all('div', class_='block pad-bottom-5')
            if fname_links:
                fname_list = [link.get_text(strip=True) for link in fname_links if link.get_text(strip=True)]
                fname = chr(10).join(fname_list) if fname_list else 'None'

        # Trích xuất người dịch
        trans = None
        id_o_l = None
        id_o = None
        trans_span = soup.find('span', class_='series-owner_name')
        if trans_span:
            trans = trans_span.string.strip()
            next_o = trans_span.find_next('a')
            if next_o and 'href' in next_o.attrs:
                id_o_l = next_o['href']
                if not id_o_l.startswith(('https://', 'http://')):
                    id_o_l = 'https://ln.hako.vn' + id_o_l
                else:
                    id_o_l = id_o_l.replace('docln.net', 'ln.hako.vn')
                    id_o = re.search(r'/(\d+)$', id_o_l)
                    id_o = id_o.group(1) if id_o else None

        # Trích xuất nhóm dịch
        team = None
        id_t_l = None
        id_t = None
        team_span = soup.find('div', class_='fantrans-value')
        if team_span:
            team = team_span.string.strip()
            next_t = team_span.find_next('a')
            if next_t and 'href' in next_t.attrs:
                id_t_l = next_t['href']
                if not id_t_l.startswith(('https://', 'http://')):
                    id_t_l = 'https://ln.hako.vn' + id_t_l
                else:
                    id_t_l = id_t_l.replace('docln.net', 'ln.hako.vn')
                id_t = re.search(r'/nhom-dich/(\d+)', id_t_l)
                id_t = id_t.group(1) if id_t else None

        # Trích xuất người tham gia
        atb = None
        id_j_l = None
        id_j = None  
        atb_span = soup.find('div', class_='series-owner_share')
        if atb_span:
            atb_links = atb_span.find_all('a', class_='ln_info-name')
            if atb_links:
                atb_list = [link.get_text(strip=True) for link in atb_links if link.get_text(strip=True)]
                atb = chr(10).join(atb_list) if atb_list else 'None'
                id_j_l_list = [link.get('href') for link in atb_links if link.get('href')]
                id_j_l_list = ['https://ln.hako.vn' + url if not url.startswith(('https://', 'http://')) else url.replace('docln.net', 'ln.hako.vn') for url in id_j_l_list]
                id_j_l = chr(10).join(id_j_l_list) if id_j_l_list else 'None'
                id_j_list = [re.search(r'/(\d+)$', url).group(1) for url in id_j_l_list if re.search(r'/(\d+)$', url)]
                id_j = chr(10).join(id_j_list) if id_j_list else 'None'

        # Trích xuất số tập
        def count_vol(soup):
            vol_spans = soup.find_all('span', class_='list_vol-title')
            return len(vol_spans)
        nvol = count_vol(soup)

        # Trích xuất số chương
        def count_chap(soup):
            chap_spans = soup.find_all('div', class_='chapter-name')
            return len(chap_spans)
        nchap = count_chap(soup)

        # Xác định hình thức dựa trên số tập và trạng thái
        format_type = "series" if nvol > 1 else (
            "Not sure" if nvol == 1 and state.lower() == "đang tiến hành" else "oneshot"
        )

        # Trích xuất thời gian bắt đầu và thời gian cập nhật mới nhất
        first_day, first_month, first_year = None, None, None
        latest_day, latest_month, latest_year = None, None, None

        chapter_time_divs = soup.find_all('div', class_='chapter-time')
        if chapter_time_divs:
            first_date_text = chapter_time_divs[0].get_text(strip=True)
            latest_date_text = chapter_time_divs[-1].get_text(strip=True)
        
            first_day, first_month, first_year = parse_date(first_date_text)
            latest_day, latest_month, latest_year = parse_date(latest_date_text)

        return (url_id,title, canonical_url, method, genres, manga, anime, cd, origin, image_link,
                author, artist, showtype, state, like, nword, rate, view, fname, id_o, 
                trans, id_o_l, id_t, team, id_t_l, id_j, atb, id_j_l, nvol, nchap, format_type,
                first_day, first_month, first_year, latest_day, latest_month, latest_year, ncom)
    
    except Exception as e:
        print(f"Error extracting info for novel {novel_id}: {str(e)}")
        return None

In [5]:
def process_novel_info(info):
    # Tách logic xử lý thông tin ra khỏi vòng lặp chính
    (url_id, title, canonical_url, method, genres, manga, anime, cd, origin, image_link,
     author, artist, showtype, state, like, nword, rate, view, fname, id_o, 
     trans, id_o_l, id_t, team, id_t_l, id_j, atb, id_j_l, nvol, nchap, format_type,
     first_day, first_month, first_year, latest_day, latest_month, latest_year, ncom) = info
    
    return {
        'ID': process_value(url_id),
        'Tựa đề': process_value(title),
        'Link hako': process_value(canonical_url),
        'Phương thức sáng tác': process_value(method),
        'Thể loại': process_value(genres),
        'Manga': process_value(manga),
        'Anime': process_value(anime),
        'CD': process_value(cd),
        'Ngôn ngữ gốc': process_value(origin),
        'Link ảnh': process_value(image_link),
        'Tác giả': process_value(author),
        'Họa sĩ': process_value(artist),
        'Loại hình': process_value(showtype),
        'Tình trạng': process_value(state),
        'Số like': convert_to_number(process_value(like, is_numeric=True)),
        'Số từ': convert_to_number(process_value(nword, is_numeric=True)),
        'Số lượt đánh giá': convert_to_number(process_value(rate, is_numeric=True)),
        'Số lượt xem': convert_to_number(process_value(view, is_numeric=True)),
        'Số lượt bình luận': convert_to_number(process_value(ncom, is_numeric=True)),
        'Fname': process_value(fname),
        'ID người dịch': process_value(id_o),
        'Người dịch': process_value(trans),
        'Link người dịch': process_value(id_o_l),
        'ID nhóm dịch': process_value(id_t),
        'Nhóm dịch': process_value(team),
        'Link nhóm dịch': process_value(id_t_l),
        'ID người tham gia': process_value(id_j),
        'Người tham gia': process_value(atb),
        'Link người tham gia': process_value(id_j_l),
        'Số tập': process_value(nvol),
        'Số chương': process_value(nchap),
        'Hình thức': process_value(format_type),
        'Ngày bắt đầu': process_value(first_day),
        'Tháng bắt đầu': process_value(first_month),   
        'Năm bắt đầu': process_value(first_year),
        'Ngày cập nhật cuối': process_value(latest_day),
        'Tháng cập nhật cuối': process_value(latest_month),
        'Năm cập nhật cuối': process_value(latest_year)
    }

In [6]:
def process_novel_batch(start_id, end_id, batch_size=100):
    all_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Chia thành các batch nhỏ
        batches = [(i, min(i + batch_size - 1, end_id)) 
                  for i in range(start_id, end_id + 1, batch_size)]
        
        futures = []
        for batch_start, batch_end in batches:
            future = executor.submit(process_batch, batch_start, batch_end)
            futures.append(future)
            
        # Theo dõi tiến độ
        total_batches = len(batches)
        completed = 0
        
        for future in as_completed(futures):
            batch_data = future.result()
            if batch_data:
                all_data.extend(batch_data)
            completed += 1
            print(f"Completed: {completed}/{total_batches} batches ({(completed/total_batches)*100:.2f}%)")
            
            # Lưu checkpoint định kỳ
            if completed % 10 == 0:  # Lưu sau mỗi 10 batch
                save_checkpoint(all_data, f'checkpoint_{completed}.xlsx')
    
    return all_data

def process_batch(start_id, end_id):
    batch_data = []
    for novel_id in range(start_id, end_id + 1):
        try:
            html_content = get_novel_html(novel_id)
            if not html_content:
                continue
                
            info = extract_info_from_html(novel_id, html_content)
            if not info:
                continue
                
            # Xử lý thông tin và thêm vào batch_data
            processed_data = process_novel_info(info)
            batch_data.append(processed_data)
            
            time.sleep(1)  # Giảm delay xuống
            
        except Exception as e:
            print(f"Error processing novel {novel_id}: {str(e)}")
            continue
    
    return batch_data

def save_checkpoint(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Saved checkpoint to {filename}")







In [7]:
# Sử dụng
if __name__ == '__main__':
    start_id = 1
    end_id = 3
    output_excel_path = 'hako_dataset_complete.csv'
    
    # Chạy với batch size 100
    all_data = process_novel_batch(start_id, end_id, batch_size=100)
    
    # Lưu kết quả cuối cùng
    df = pd.DataFrame(all_data)
    df.to_csv(output_excel_path, index=False, encoding='utf-8-sig')
    print(f"Completed! Saved {len(all_data)} novels to {output_excel_path}")

Completed: 1/1 batches (100.00%)
Completed! Saved 3 novels to hako_dataset_complete.csv
